In [1]:
# Adapted from https://le.qun.ch/en/blog/2023/05/13/transformer-batching/
import itertools
import numpy as np
import pandas as pd
import pickle
import gzip
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
import torch
torch.set_grad_enabled(False)
import transformers
from tqdm import tqdm
import intrasm_engine
from benchmark.utils_bench_transformer import gen_opt_cfg, benchmark_dense, benchmark_qk_ar, benchmark_qk_init, time_greedy_generate

Print handle to force cublas initialization (otherwise first matmul captured in the graph may fail): 190660736
Setting float16 and bf16 using reduced precision in reduction


In [3]:
optcfg = gen_opt_cfg()

In [4]:
#nd_list = list(itertools.chain(itertools.product([12, 16, 32], [64]), itertools.product([32, 40, 56, 72, 96], [128])))
#seqlen_list = [10, 20, 50, 100, 200, 500, 1000, 2000, 5000]
# Reducing the size to fit in RTX 3090 device memory
nd_list = list(itertools.chain(itertools.product([12, 16, 32], [64]), itertools.product([32, 40, 56], [128])))
seqlen_list = [10, 20, 50, 100, 200, 500, 1000]
bs_list = list(itertools.chain(range(1, 8), range(8, 16, 2), range(16, 32, 4), range(32, 64, 8), range(64, 128, 16), [128]))

In [5]:
print(nd_list)
print(seqlen_list)
print(bs_list)

[(12, 64), (16, 64), (32, 64), (32, 128), (40, 128), (56, 128)]
[10, 20, 50, 100, 200, 500, 1000]
[1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 14, 16, 20, 24, 28, 32, 40, 48, 56, 64, 80, 96, 112, 128]


In [6]:
data = {}

In [7]:
db = []
benchmark_qk_init(db, nd_list, seqlen_list, bs_list)
data["qk_init"] = db

  0%|          | 0/1008 [00:00<?, ?it/s, bs=128, d=128, h=7168, n=56, seqlen=1000]

Working on 56 128 1000


  2%|▏         | 24/1008 [00:48<32:51,  2.00s/it, bs=128, d=128, h=7168, n=56, seqlen=500]

Working on 56 128 500


  5%|▍         | 48/1008 [01:36<32:01,  2.00s/it, bs=128, d=128, h=7168, n=56, seqlen=200]

...

 95%|█████████▌| 960/1008 [32:03<01:36,  2.00s/it, bs=128, d=64, h=768, n=12, seqlen=20]

Working on 12 64 20


 98%|█████████▊| 984/1008 [32:51<00:48,  2.00s/it, bs=128, d=64, h=768, n=12, seqlen=10]

Working on 12 64 10


100%|██████████| 1008/1008 [33:39<00:00,  2.00s/it, bs=1, d=64, h=768, n=12, seqlen=10] 


In [ ]:
db = []
benchmark_qk_ar(db, nd_list, seqlen_list, bs_list)
data["qk_ar"] = db

In [ ]:
db = []
benchmark_dense(db, nd_list, seqlen_list, bs_list)
data["dense"] = db

In [ ]:
with gzip.open("data/20230516-transformer-batching.pkl.gz", "wb") as f:
    pickle.dump(data, f)

In [ ]:
df_dense = (
    pd.DataFrame.from_dict(data["dense"])
    .assign(h=lambda x: x["n"] * x["d"])
    .assign(flop=lambda x: (x["bs"] * x["seqlen"] * x["h"]**2) * 2)
    .assign(io=lambda x: (x["bs"]*x["seqlen"]*x["h"]*2 + x["h"]**2) * 2)
    .assign(intensity=lambda x: x["flop"] / x["io"])
    .assign(throughput=lambda x: x["bs"]*x["seqlen"] / x["latency"])
    .assign(series="dense")
)
df_qk_init = (
    pd.DataFrame.from_dict(data["qk_init"])
    .assign(h=lambda x: x["n"] * x["d"])
    .assign(flop=lambda x: (x["bs"]*x["n"]*x["d"]*x["seqlen"]**2) * 2)
    .assign(io=lambda x: (x["bs"]*x["n"]*(x["seqlen"]*x["d"]*2 + x["seqlen"]**2)) * 2)
    .assign(intensity=lambda x: x["flop"] / x["io"])
    .assign(throughput=lambda x: x["bs"]*x["seqlen"] / x["latency"])
    .assign(series="qk_init")
)
df_qk_ar = (
    pd.DataFrame.from_dict(data["qk_ar"])
    .assign(h=lambda x: x["n"] * x["d"])
    .assign(flop=lambda x: (x["bs"]*x["n"]*x["d"]*x["seqlen"]) * 2)
    .assign(io=lambda x: (x["bs"]*x["n"]*(x["d"] + x["seqlen"]*x["d"] + x["seqlen"])) * 2)
    .assign(intensity=lambda x: x["flop"] / x["io"])
    .assign(throughput=lambda x: x["bs"] / x["latency"])
    .assign(series="qk_ar")
)
pd.concat([df_dense, df_qk_init, df_qk_ar]).to_csv("data/transformer-batching-microbenchmarks.csv", index=False)

In [ ]:
opt_config = optcfg["6.7b"]

torch.set_default_dtype(torch.bfloat16)
with transformers.modeling_utils.no_init_weights():
    model = transformers.models.opt.OPTForCausalLM(opt_config).to("cuda")
torch.set_default_dtype(torch.float32)

In [ ]:
db = {}
input_tokens = 200
new_tokens = 500
for bs in tqdm(list(itertools.chain(range(1, 8), range(8, 16, 2), [16]))):
    x = torch.randint(1000, 10000, (bs, input_tokens), device=model.device)
    stack = []
    for _ in range(10):
        l = time_greedy_generate(model, x, new_tokens=new_tokens)
        stack.append(l)
    db[bs] = np.median(np.stack(stack), axis=0)
    del x
    torch.cuda.empty_cache()
del model
torch.cuda.empty_cache()

with gzip.open("data/20230516-e2e-text-generation-batch.pkl.gz", "wb") as f:
    pickle.dump(db, f)